# Advent of code 2022

In [1]:
import pandas as pd

## 01

### Code 

In [2]:
def read_txt_file(path: str):
    with open(path) as f:
        string = f.read()
    return string


def solve_01(path: str, n=1):
    input_txt = read_txt_file(path)

    elf_food_sums = []
    for elf_food_str in input_txt.split("\n\n"):
        elf_food_int = [int(x) for x in elf_food_str.split("\n") if x != ""]
        elf_food_sums.append(sum(elf_food_int))
    return sum(sorted(elf_food_sums)[-n:])

### Example

In [3]:
example_path = "inputs/01_example.txt"

In [4]:
solve_01(example_path)

24000

In [5]:
solve_01(example_path, n=3)

45000

### Puzzle

In [6]:
input_file = "inputs/01_input.txt"

In [7]:
solve_01(input_file)

66616

In [8]:
solve_01(input_file, n=3)

199172

## 02

### Code part 1

In [9]:
def compute_outcome(df):
    df.loc[(df["me"] == df["other"]), "outcome"] = "draw"

    df.loc[(df["me"] == "rock") & (df["other"] == "paper"), "outcome"] = "lose"
    df.loc[(df["me"] == "rock") & (df["other"] == "scissors"), "outcome"] = "win"

    df.loc[(df["me"] == "paper") & (df["other"] == "rock"), "outcome"] = "win"
    df.loc[(df["me"] == "paper") & (df["other"] == "scissors"), "outcome"] = "lose"

    df.loc[(df["me"] == "scissors") & (df["other"] == "paper"), "outcome"] = "win"
    df.loc[(df["me"] == "scissors") & (df["other"] == "rock"), "outcome"] = "lose"

    return df["outcome"]


def solve_02_part1(path: str):
    input_txt = read_txt_file(path)
    rounds_str = []
    for strategy_str in input_txt.split("\n"):
        rounds_str.append(strategy_str.split(" "))

    # Define rounds
    rounds = pd.DataFrame(rounds_str, columns=["other", "me"])
    rounds["other"] = rounds["other"].map({"A": "rock", "B": "paper", "C": "scissors"})
    rounds["me"] = rounds["me"].map({"X": "rock", "Y": "paper", "Z": "scissors"})

    # Score strategy
    rounds = rounds.assign(outcome=compute_outcome)

    outcome_points = {"win": 6, "draw": 3, "lose": 0}
    rounds["outcome_points"] = rounds["outcome"].map(outcome_points)

    outcome_points = {"rock": 1, "paper": 2, "scissors": 3}
    rounds["shape_points"] = rounds["me"].map(outcome_points)

    rounds["points"] = rounds["outcome_points"] + rounds["shape_points"]

    return rounds["points"].sum()

### Code part 2

In [10]:
def compute_my_move(df):
    df.loc[(df["outcome"] == "draw"), "me"] = df["other"]

    df.loc[(df["outcome"] == "lose") & (df["other"] == "rock"), "me"] = "scissors"
    df.loc[(df["outcome"] == "lose") & (df["other"] == "paper"), "me"] = "rock"
    df.loc[(df["outcome"] == "lose") & (df["other"] == "scissors"), "me"] = "paper"

    df.loc[(df["outcome"] == "win") & (df["other"] == "rock"), "me"] = "paper"
    df.loc[(df["outcome"] == "win") & (df["other"] == "paper"), "me"] = "scissors"
    df.loc[(df["outcome"] == "win") & (df["other"] == "scissors"), "me"] = "rock"

    return df["me"]


def solve_02_part2(path: str):
    # Loading the inputs
    input_txt = read_txt_file(path)
    rounds_str = []
    for strategy_str in input_txt.split("\n"):
        rounds_str.append(strategy_str.split(" "))

    # Define rounds
    rounds = pd.DataFrame(rounds_str, columns=["other", "outcome"])
    rounds["other"] = rounds["other"].map({"A": "rock", "B": "paper", "C": "scissors"})
    rounds["outcome"] = rounds["outcome"].map({"X": "lose", "Y": "draw", "Z": "win"})

    # Score strategy
    rounds = rounds.assign(me=compute_my_move)

    outcome_points = {"win": 6, "draw": 3, "lose": 0}
    rounds["outcome_points"] = rounds["outcome"].map(outcome_points)

    outcome_points = {"rock": 1, "paper": 2, "scissors": 3}
    rounds["shape_points"] = rounds["me"].map(outcome_points)

    rounds["points"] = rounds["outcome_points"] + rounds["shape_points"]

    return rounds["points"].sum()


example_path = "inputs/02_example.txt"
solve_02_part2(example_path)

12

### Example

In [11]:
example_path = "inputs/02_example.txt"

In [12]:
solve_02_part1(example_path)

15

In [13]:
solve_02_part2(example_path)

12

### Puzzle

In [14]:
input_file = "inputs/02_input.txt"

In [15]:
solve_02_part1(input_file)

15523.0

In [16]:
solve_02_part2(input_file)

15702.0

## 03

### Code

In [17]:
import string

item_priorities = {x: i + 1 for i, x in enumerate(string.ascii_letters)}


class Rucksack:
    def __init__(self, content: str):
        self.content = content
        # print(self.content)
        self.nb_items = len(self.content)
        split = int(self.nb_items / 2)
        self.content_set_1 = set(self.content[:split])
        self.content_set_2 = set(self.content[split:])

    def get_common_item_priority(self):
        try:
            commom_item = list(self.content_set_1 & self.content_set_2)[0]
            return item_priorities[commom_item]
        except IndexError:
            return 0


def solve_03_part1(path: str):
    input_txt = read_txt_file(path)
    rucksack_contents = input_txt.split("\n")

    common_item_priorities = []
    for rucksack_content in rucksack_contents:
        r = Rucksack(rucksack_content)
        common_item_priorities.append(r.get_common_item_priority())

    return sum(common_item_priorities)


# Run some tests
example_file = "inputs/03_example.txt"
assert solve_03_part1(example_file) == 157

input_file = "inputs/03_input.txt"
assert solve_03_part1(input_file) == 8123


def solve_03_part2(path: str):
    input_txt = read_txt_file(path)
    rucksack_contents = input_txt.split("\n")

    nb_teams = int(len(rucksack_contents) / 3)
    common_item_priorities = []
    for i in range(nb_teams):
        team_rucksacks = rucksack_contents[i * 3 : (i + 1) * 3]
        team_rucksacks_sets = [set(x) for x in team_rucksacks]
        try:
            commom_item = list(
                team_rucksacks_sets[0] & team_rucksacks_sets[1] & team_rucksacks_sets[2]
            )[0]
            common_item_priorities.append(item_priorities[commom_item])
        except IndexError:
            pass
    return sum(common_item_priorities)

### Example

In [18]:
example_file = "inputs/03_example.txt"

In [19]:
solve_03_part1(example_file)

157

In [20]:
solve_03_part2(example_file)

70

### Puzzle

In [21]:
input_file = "inputs/03_input.txt"

In [22]:
solve_03_part1(input_file)

8123

In [23]:
solve_03_part2(input_file)

2620

## 04

### Code

In [38]:
class AssignmentPairs:
    def __init__(self, input_str: str):
        self.input_str = input_str
        # print(self.input_str)
        self.assignments_str = self.input_str.split(",")
        self.assignments_set_1 = self._create_assignment_sets(self.assignments_str[0])
        self.assignments_set_2 = self._create_assignment_sets(self.assignments_str[1])
        # print(self.assignments_set_1, self.assignments_set_2)

    def _create_assignment_sets(self, input_str: str):
        start, end = [int(x) for x in input_str.split("-")]
        return set(range(start, end + 1))

    def is_fully_contained(self):
        cond_1 = self.assignments_set_1.issubset(self.assignments_set_2)
        cond_2 = self.assignments_set_2.issubset(self.assignments_set_1)
        # print(cond_1 | cond_2)
        return cond_1 | cond_2

    def has_overlap(self):
        # print(self.assignments_set_1 & self.assignments_set_2)
        return len(self.assignments_set_1 & self.assignments_set_2) > 0

def solve_04(path: str, part=1):
    input_txt = read_txt_file(path)
    assignments_str = input_txt.split("\n")
    try:
        assignments_str.remove("")
    except ValueError:
        pass

    results = []
    for assignment_str in assignments_str:
        ap = AssignmentPairs(assignment_str)
        if part == 1:
            results.append(ap.is_fully_contained())
        else:
            results.append(ap.has_overlap())

    return sum(results)


# Run some tests
example_file = "inputs/04_example.txt"
assert solve_04(example_file) == 2

input_file = "inputs/04_input.txt"
assert solve_04(input_file) == 540

4

### Example

In [25]:
example_file = "inputs/04_example.txt"

In [26]:
solve_04(example_file)

2

In [39]:
solve_04(example_file, part=2)

4

### Puzzle

In [28]:
input_file = "inputs/04_input.txt"

In [29]:
solve_04(input_file)

540

In [40]:
solve_04(input_file, part=2)

872